In [12]:
import os
import numpy as np
import matplotlib.pyplot as plt
import glob
from pydub import AudioSegment
from pydub.silence import split_on_silence
from scipy import signal
from scipy.io import wavfile

### Usuwanie ciszy
Funkcja remove_silence() iteruje po ścieżkach dźwiękowych w folderze '/Data/' i dodaje do podfolderu '/Edited/' te same ścieżki dźwiękowe z usuniętą ciszą.

In [15]:
def remove_silence():
    Output_folder = 'Data/Edited' 
    if not os.path.exists(Output_folder):
        os.makedirs(Output_folder)
    for file_name in glob.glob("Data/*.wav"):
        audio_format = "wav"
        output_name = 'Data/Edited/'+file_name.split('\\')[1].split('.')[0]+'_Edited.wav'
        print(file_name)
        print(output_name)

        # Reading and splitting the audio file into chunks
        sound = AudioSegment.from_file(file_name, format = audio_format) 
        audio_chunks = split_on_silence(sound
                                    ,min_silence_len = 100
                                    ,silence_thresh = -45
                                    ,keep_silence = 50
                                )
        # Putting the file back together
        combined = AudioSegment.empty()
        for chunk in audio_chunks:
            combined += chunk
        combined.export(output_name, format = audio_format)

In [16]:
remove_silence()

Data\f1_script1_cleanraw.wav
Data/Edited/f1_script1_cleanraw_Edited.wav
Data\m10_script5_cleanraw.wav
Data/Edited/m10_script5_cleanraw_Edited.wav


### Dzielenie ścieżek na fragmenty
Funkcja cut_data() iteruje po folderze "/Data/Edited" i każdą ścieżkę dźwiękową (z usuniętą ciszą) dzieli na 2 sekundowe fragmenty i zapisuje w folderze "/Data/Cut"

In [18]:
# Saves segments into .wav files
def segments_to_wav(segments, filename, output_folder, sample_rate):
    for i, segment in enumerate(segments):
        path = os.path.join(output_folder,f"{filename}_part_{i+1}.wav")
        wavfile.write(path, sample_rate, segment)

In [19]:
# Splits good audio parts into segments of specified duration ready to be saved
def split_audio_to_segments(data, sample_rate, segment_duration = 2):

    segment_samples = int(segment_duration*sample_rate)
    segments = []

    for i in range(len(data)//segment_samples):
        segment = data[i * segment_samples:(i + 1) * segment_samples]
        segments.append(segment)

    return segments

In [20]:
def cut_file(filename, input_folder, output_folder, segments_duration = 2, threshold = 400):

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    file_path = os.path.join(input_folder, filename)
    base_name = os.path.splitext(os.path.basename(filename))[0]

    sample_rate, data = wavfile.read(file_path)
    
    # Checking if stereo (using only mono)
    if len(data.shape) == 2:
        data = data[:0]

    segments = split_audio_to_segments(data, sample_rate, segments_duration)

    segments_to_wav(segments, base_name, output_folder, sample_rate)

In [21]:
for file in os.listdir('Data/Edited'):
    if(file.endswith(".wav")):
        cut_file(file, 'Data/Edited', 'Data/Cut')

### Tworzenie spektrogramów
Z każdej 2 sekundowej ścieżki dźwiękowej tworzony jest spektrogram i zapisywany jest w folderze '/Spectrograms/'

In [23]:
def create_spectrogram(file, ofolder_path, ifolder_path):
    
    file_path = os.path.join(ifolder_path, file)

    sample_rate, samples = wavfile.read(file_path)
    frequencies, times, spectrogram = signal.spectrogram(samples, sample_rate)

    base_name = os.path.splitext(os.path.basename(file))[0]

    plt.figure(figsize=(10,5))
    #Deleting cmap='gray' speeds the creation of spectrogram by 5 times, but its colorful
    plt.pcolormesh(times, frequencies, np.log(spectrogram), shading='gouraud', cmap='gray')
    plt.title(base_name)
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [sec]')

    new_path = os.path.join(ofolder_path, base_name + '.jpg')

    plt.savefig(new_path)
    plt.close()

In [24]:
for file in os.listdir('Data/Cut'):
    if(file.endswith(".wav")):
        create_spectrogram(file, 'Spectrograms', 'Data/Cut')
